In [1]:
from transformers import pipeline
import pandas as pd

C:\Users\tighu\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\tighu\AppData\Local\Temp\ipykernel_23364\183117859.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [27]:
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli", device= 0)
movie_reviews_df = pd.read_csv("IMDB Dataset.csv")

In [16]:
movie_reviews_df.shape

(50000, 2)

In [10]:
# use zero shot classifier for the three labels
candidate_labels = ['positive', 'negative', 'neutral']

llm_label = []
llm_confidence = []
count = 0
for review in movie_reviews_df['review']:
    
    zero_shot_result = classifier(review, candidate_labels)
    
    llm_label.append(zero_shot_result['labels'][0])
    llm_confidence.append(zero_shot_result['scores'][0])  

    count +=1

    if count % 1000 == 0: print(count)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000


In [28]:
movie_reviews_df["llm_label_bert"] = llm_label
movie_reviews_df["llm_confidence_bert"] = llm_confidence

In [29]:
movie_reviews_df.head(5)

,review,sentiment,llm_label_bert,llm_confidence_bert
0,One of the other reviewers has mentioned that ...,positive,positive,0.410353
1,A wonderful little production. <br /><br />The...,positive,positive,0.948007
2,I thought this was a wonderful way to spend ti...,positive,positive,0.933053
3,Basically there's a family where a little boy ...,negative,negative,0.950203
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,positive,0.768185


In [30]:
movie_reviews_df["sentiment"].value_counts()
#even split to begin with initial dataset

sentiment
positive    25000
negative    25000
Name: count, dtype: int64

In [31]:
# split based on BERT LLM
# introduction of minority class neutral 250:  0.5%
movie_reviews_df["llm_label_bert"].value_counts()

llm_label_bert
negative    27149
positive    22610
neutral       241
Name: count, dtype: int64

In [39]:
# all the rows where two are unequal
# it accounts for around 12% of the entire dataset
label_errors_df = movie_reviews_df.query('llm_label_bert != sentiment')

In [53]:
# putting some of the cases where a new sentiment was assigned to the movie rewiew

label_errors_df.head(5)

,review,sentiment,llm_label_bert,llm_confidence_bert
16,Some films just simply should not be remade. T...,positive,negative,0.811583
25,The Karen Carpenter Story shows a little more ...,positive,negative,0.761770
29,'War movie' is a Hollywood genre that has been...,positive,negative,0.601619
30,"Taut and organically gripping, Edward Dmytryk'...",positive,negative,0.753970
31,"""Ardh Satya"" is one of the finest film ever ma...",positive,negative,0.636056


In [60]:
label_errors_df['review'][16]

# Some films just simply should not be remade. This is one of them. In and of itself it is not a bad film. But it fails to capture the flavor and 
# the terror of the 1963 film of the same title. Liam Neeson was excellent as he always is, and most of the cast holds up, with the exception of Owen Wilson, 
# who just did not bring the right feel to the character of Luke. But the major fault with this version is that it strayed too far from the Shirley Jackson 
# story in it's attempts to be grandiose and lost some of the thrill of the earlier film in a trade off for snazzier special effects. Again I will say that 
# in and of itself it is not a bad film. But you will enjoy the friction of terror in the older version much more.

Some films just simply should not be remade. This is one of them. In and of itself it is not a bad film. But it fails to capture the flavor and the terror of the 1963 film of the same title. Liam Neeson was excellent as he always is, and most of the cast holds up, with the exception of Owen Wilson, who just did not bring the right feel to the character of Luke. But the major fault with this version is that it strayed too far from the Shirley Jackson story in it's attempts to be grandiose and lost some of the thrill of the earlier film in a trade off for snazzier special effects. Again I will say that in and of itself it is not a bad film. But you will enjoy the friction of terror in the older version much more.


In [64]:
label_errors_df['review'][25]
# "The Karen Carpenter Story shows a little more about singer Karen Carpenter's complex life. Though it fails in giving accurate facts, and details.<br /><br 
# />Cynthia Gibb (portrays Karen) was not a fine election. She is a good actress , but plays a very naive and sort of dumb Karen Carpenter. I think that the 
# role needed a stronger character. Someone with a stronger personality.<br /><br />Louise Fletcher role as Agnes Carpenter is terrific, she does a great job 
# as Karen's mother.<br /><br />It has great songs, which could have been included in a soundtrack album. Unfortunately they weren't, though this movie was on 
# the top of the ratings in USA and other several countries"

"The Karen Carpenter Story shows a little more about singer Karen Carpenter's complex life. Though it fails in giving accurate facts, and details.<br /><br />Cynthia Gibb (portrays Karen) was not a fine election. She is a good actress , but plays a very naive and sort of dumb Karen Carpenter. I think that the role needed a stronger character. Someone with a stronger personality.<br /><br />Louise Fletcher role as Agnes Carpenter is terrific, she does a great job as Karen's mother.<br /><br />It has great songs, which could have been included in a soundtrack album. Unfortunately they weren't, though this movie was on the top of the ratings in USA and other several countries"

In [ ]:
# Although both reviews were negatively leaning they were initially assigned a positive label in the dataset. Number of such rewviews in the dataset amount
# to 6K which is about 12% of the entire dataset. So any model trained with the original dataset would have skewed trained and test set which would
# severly impact its working in the real world. In this excercise zero shot classifier in form of Bert Large is used to improve accuracy of dataset by 12% and
# adding more diversity to reviews labels by adding a neutral label.